<a href="https://colab.research.google.com/github/abhisheks666/hello-world/blob/master/Yelp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
#Lets move to the place where our BA files are stored
%cd /content/drive/MyDrive/BA2023/

Mounted at /content/drive/
/content/drive/MyDrive/BA2023


In [ ]:
%cd /content/drive/MyDrive/BA2023/NLP

/content/drive/MyDrive/BA2023/NLP


In [ ]:
!ls

yelp_academic_dataset_review.json  Yelp.csv  YelpReviews.csv


In [ ]:
import gensim
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
import string
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Convert json to csv
input_json_file = 'yelp_academic_dataset_review.json'
input_csv_file = 'Yelp.csv'
output_csv_file = 'YelpReviews.csv'

# Load only the first 100,000 rows from the JSON file into a Pandas DataFrame
df = pd.read_json(input_json_file, lines=True, nrows=100000)

# Convert the DataFrame to CSV
df.to_csv(input_csv_file, index=False)
columns_to_keep = ['stars', 'text']

# Create a new DataFrame with only the specified columns
df_selected_columns = df[columns_to_keep]

# Write the new DataFrame to a new CSV file
df_selected_columns.to_csv(output_csv_file, index=False)

print(f'New CSV file created: {output_csv_file}')


New CSV file created: YelpReviews.csv


In [ ]:
# read csv file into pandas using a relative path
yelp_reviews = pd.read_csv('YelpReviews.csv')

In [ ]:
yelp_reviews.head()

,stars,text
0,3,"If you decide to eat here, just be aware it is..."
1,5,I've taken a lot of spin classes over the year...
2,3,Family diner. Had the buffet. Eclectic assortm...
3,5,"Wow! Yummy, different, delicious. Our favo..."
4,4,Cute interior and owner (?) gave us tour of up...


**We are assuming that a rating of 4 or above is a good review and a rating of 3 or less is a bad review**

In [ ]:
# pos for good reviews (rating of 4 or above) and neg for bad reviews (rating of 3 or below)
yelp_reviews['rating'] = yelp_reviews['stars'].apply(lambda c: 'pos' if c >=4 else 'neg')

In [ ]:
yelp_reviews.head()

,stars,text,rating
0,3,"If you decide to eat here, just be aware it is...",neg
1,5,I've taken a lot of spin classes over the year...,pos
2,3,Family diner. Had the buffet. Eclectic assortm...,neg
3,5,"Wow! Yummy, different, delicious. Our favo...",pos
4,4,Cute interior and owner (?) gave us tour of up...,pos


In [ ]:
duplicate_values = yelp_reviews['text'].duplicated()
print(duplicate_values)

0        False
1        False
2        False
3        False
4        False
         ...  
99995    False
99996    False
99997    False
99998    False
99999    False
Name: text, Length: 100000, dtype: bool


In [ ]:
yelp_reviews.drop_duplicates(subset=['text'], inplace=True)

In [ ]:
yelp_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99964 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   stars   99964 non-null  int64 
 1   text    99964 non-null  object
 2   rating  99964 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.1+ MB


In [ ]:
yelp_reviews.isnull().sum()

stars     0
text      0
rating    0
dtype: int64

In [ ]:
yelp_reviews['text'].fillna(' ', inplace=True)

In [ ]:
import nltk

nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
sid.polarity_scores("He not bad in the movie.")

{'neg': 0.0, 'neu': 0.637, 'pos': 0.363, 'compound': 0.431}

In [ ]:
yelp_reviews.head()

,stars,text,rating
0,3,"If you decide to eat here, just be aware it is...",neg
1,5,I've taken a lot of spin classes over the year...,pos
2,3,Family diner. Had the buffet. Eclectic assortm...,neg
3,5,"Wow! Yummy, different, delicious. Our favo...",pos
4,4,Cute interior and owner (?) gave us tour of up...,pos


In [ ]:
yelp_reviews['scores'] = yelp_reviews['text'].apply(lambda review: sid.polarity_scores(review))

yelp_reviews['compound']  = yelp_reviews['scores'].apply(lambda score_dict: score_dict['compound'])

yelp_reviews['comp_score'] = yelp_reviews['compound'].apply(lambda c: 'pos' if c >=0.80 else 'neg')

In [ ]:
yelp_reviews

,stars,text,rating,scores,compound,comp_score
0,3,"If you decide to eat here, just be aware it is...",neg,"{'neg': 0.0, 'neu': 0.888, 'pos': 0.112, 'comp...",0.8597,pos
1,5,I've taken a lot of spin classes over the year...,pos,"{'neg': 0.05, 'neu': 0.705, 'pos': 0.245, 'com...",0.9858,pos
2,3,Family diner. Had the buffet. Eclectic assortm...,neg,"{'neg': 0.035, 'neu': 0.709, 'pos': 0.257, 'co...",0.9201,pos
3,5,"Wow! Yummy, different, delicious. Our favo...",pos,"{'neg': 0.0, 'neu': 0.66, 'pos': 0.34, 'compou...",0.9588,pos
4,4,Cute interior and owner (?) gave us tour of up...,pos,"{'neg': 0.017, 'neu': 0.711, 'pos': 0.272, 'co...",0.9804,pos
...,...,...,...,...,...,...
99995,4,Came here for lunch with a group. They were bu...,pos,"{'neg': 0.0, 'neu': 0.749, 'pos': 0.251, 'comp...",0.9144,pos
99996,1,The equipment is so old and so felty! I just u...,neg,"{'neg': 0.058, 'neu': 0.814, 'pos': 0.129, 'co...",0.4519,neg
99997,4,This is one of my favorite Mexican restaurants...,pos,"{'neg': 0.068, 'neu': 0.831, 'pos': 0.1, 'comp...",0.0772,neg
99998,2,Came here for brunch - had an omlette ($19 + t...,neg,"{'neg': 0.0, 'neu': 0.873, 'pos': 0.127, 'comp...",0.5859,neg


In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
accuracy_score(yelp_reviews['rating'],yelp_reviews['comp_score'])

0.7795306310271698

In [ ]:
print(classification_report(yelp_reviews['rating'],yelp_reviews['comp_score']))

              precision    recall  f1-score   support

         neg       0.64      0.64      0.64     30262
         pos       0.84      0.84      0.84     69702

    accuracy                           0.78     99964
   macro avg       0.74      0.74      0.74     99964
weighted avg       0.78      0.78      0.78     99964



In [ ]:
print(confusion_matrix(yelp_reviews['rating'],yelp_reviews['comp_score']))

[[19269 10993]
 [11046 58656]]


So, it looks like VADER could more or less judge the Yelp reviews accurately. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a sentiment analysis tool that is specifically designed to analyze text data and determine the sentiment polarity of a piece of text. The sentiment is typically categorized into positive, negative, or neutral. In your case, you mentioned that we obtained an accuracy score of 78% for VADER analysis on Yelp reviews.
We must keep in mind that sentiment analysis models, including VADER, may perform differently depending on the domain and context. Yelp reviews might contain specific language, slang, or context that impacts sentiment analysis results.
It's always beneficial to explore and analyze some misclassifications. We can look at examples where VADER made incorrect predictions and try to understand the reasons behind them. This analysis can provide insights into areas for improvement or model fine-tuning.

**MODELS**

In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package bcp47 to /root/nltk_data...
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Un


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

**MULTINOMIAL NAIVE BAYES**

In [ ]:
X = yelp_reviews.text
y = yelp_reviews.rating

# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# instantiate the vectorizer
vect = TfidfVectorizer(max_features=10000)

X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

print(X_train_dtm)

  (0, 1011)	0.3002405488503734
  (0, 3199)	0.21899325515359835
  (0, 5312)	0.24502799708184103
  (0, 4462)	0.386711196361719
  (0, 8937)	0.0663392378590813
  (0, 9903)	0.22388150194264267
  (0, 442)	0.06725553213407312
  (0, 3478)	0.3293644983813699
  (0, 4722)	0.09325087202376209
  (0, 9065)	0.31829194631678553
  (0, 718)	0.3516185288673672
  (0, 8357)	0.2406095503548159
  (0, 1273)	0.2885131083242293
  (0, 2391)	0.30291348136448265
  (0, 9547)	0.13511022475728351
  (1, 7189)	0.06306902507291931
  (1, 4278)	0.07637241797124572
  (1, 9037)	0.04663688438845059
  (1, 5282)	0.07053969240493135
  (1, 7163)	0.050874189372505305
  (1, 5970)	0.0541776467042769
  (1, 8441)	0.09778782442468259
  (1, 3703)	0.06479689851995363
  (1, 7176)	0.09445226065322085
  (1, 9714)	0.039211602685138486
  :	:
  (74972, 5995)	0.08093489741291121
  (74972, 6030)	0.05605715138808349
  (74972, 959)	0.07922309408437042
  (74972, 7846)	0.06381888410774725
  (74972, 4255)	0.06467154414781723
  (74972, 6154)	0.070221

In [ ]:
print(X.shape)

(99964,)


In [ ]:
print(y.shape)

(99964,)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

# train the model using X_train_dtm
nb.fit(X_train_dtm, y_train)

# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

y_pred_train = nb.predict(X_train_dtm)

# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_train, y_pred_train))
print(metrics.accuracy_score(y_test, y_pred_class))

# print the confusion matrix for Train
print(metrics.confusion_matrix(y_train, y_pred_train))
print('---------------------')
print(metrics.classification_report(y_train, y_pred_train))

# print the confusion matrix for Test
print(metrics.confusion_matrix(y_test, y_pred_class))
print('------------------------')
print(metrics.classification_report(y_test, y_pred_class))

0.8680858442372588
0.8602696970909527
[[14204  8441]
 [ 1449 50879]]
---------------------
              precision    recall  f1-score   support

         neg       0.91      0.63      0.74     22645
         pos       0.86      0.97      0.91     52328

    accuracy                           0.87     74973
   macro avg       0.88      0.80      0.83     74973
weighted avg       0.87      0.87      0.86     74973

[[ 4620  2997]
 [  495 16879]]
------------------------
              precision    recall  f1-score   support

         neg       0.90      0.61      0.73      7617
         pos       0.85      0.97      0.91     17374

    accuracy                           0.86     24991
   macro avg       0.88      0.79      0.82     24991
weighted avg       0.87      0.86      0.85     24991



In [ ]:
print("Multinomial Naive Bayes Classification Accuracy:",metrics.accuracy_score(y_test, y_pred_class))

Multinomial Naive Bayes Classification Accuracy: 0.8602696970909527


**LOGISTIC REGRESSION**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# instantiate the vectorizer
vect = TfidfVectorizer(max_features=10000)

X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

X_train_dtm

<79971x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 5144027 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

# train the model using X_train_dtm
lr.fit(X_train_dtm, y_train)

# make class predictions for X_test_dtm
y_pred_class = lr.predict(X_test_dtm)

y_pred_train = lr.predict(X_train_dtm)

# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_train, y_pred_train))
print(metrics.accuracy_score(y_test, y_pred_class))

# print the confusion matrix for Train
print(metrics.confusion_matrix(y_train, y_pred_train))
print('---------------------')
print(metrics.classification_report(y_train, y_pred_train))

# print the confusion matrix for Test
print(metrics.confusion_matrix(y_test, y_pred_class))
print('------------------------')
print(metrics.classification_report(y_test, y_pred_class))

0.9167823335959285
0.9020657230030511
[[19879  4331]
 [ 2324 53437]]
---------------------
              precision    recall  f1-score   support

           0       0.90      0.82      0.86     24210
           1       0.93      0.96      0.94     55761

    accuracy                           0.92     79971
   macro avg       0.91      0.89      0.90     79971
weighted avg       0.92      0.92      0.92     79971

[[ 4811  1241]
 [  717 13224]]
------------------------
              precision    recall  f1-score   support

           0       0.87      0.79      0.83      6052
           1       0.91      0.95      0.93     13941

    accuracy                           0.90     19993
   macro avg       0.89      0.87      0.88     19993
weighted avg       0.90      0.90      0.90     19993



In [ ]:
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, y_pred_class))

Logistic Regression Accuracy: 0.9020657230030511


**Thus, we can see that the Logistic Regression model gives us the best results.**

**RANDOM FOREST**

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# instantiate the vectorizer
vect = TfidfVectorizer(max_features=1000)

X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

X_train_dtm

<74973x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 3839689 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

# train the model using X_train_dtm
rf.fit(X_train_dtm, y_train)

# make class predictions for X_test_dtm
y_pred_class = rf.predict(X_test_dtm)

y_pred_train = rf.predict(X_train_dtm)

# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_train, y_pred_train))
print(metrics.accuracy_score(y_test, y_pred_class))

# print the confusion matrix for Train
print(metrics.confusion_matrix(y_train, y_pred_train))
print('---------------------')
print(metrics.classification_report(y_train, y_pred_train))

# print the confusion matrix for Test
print(metrics.confusion_matrix(y_test, y_pred_class))
print('------------------------')
print(metrics.classification_report(y_test, y_pred_class))

0.999986661864938
0.8590692649353767
[[22644     1]
 [    0 52328]]
---------------------
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     22645
           1       1.00      1.00      1.00     52328

    accuracy                           1.00     74973
   macro avg       1.00      1.00      1.00     74973
weighted avg       1.00      1.00      1.00     74973

[[ 4545  3072]
 [  450 16924]]
------------------------
              precision    recall  f1-score   support

           0       0.91      0.60      0.72      7617
           1       0.85      0.97      0.91     17374

    accuracy                           0.86     24991
   macro avg       0.88      0.79      0.81     24991
weighted avg       0.87      0.86      0.85     24991



In [ ]:
print("Random Forest Accuracy:",metrics.accuracy_score(y_test, y_pred_class))

Random Forest Accuracy: 0.8631107198591493


**XGBoost**

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# instantiate the vectorizer
vect = TfidfVectorizer(max_features=10000)

X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

X_train_dtm

<74973x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 4819607 stored elements in Compressed Sparse Row format>

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

xgb = XGBClassifier()

# train the model using X_train_dtm
xgb.fit(X_train_dtm, y_train)

# make class predictions for X_test_dtm
y_pred_class = xgb.predict(X_test_dtm)
y_pred_class = le.fit_transform(y_pred_class)

y_pred_train = xgb.predict(X_train_dtm)
y_pred_train = le.fit_transform(y_pred_train)

# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_train, y_pred_train))
print(metrics.accuracy_score(y_test, y_pred_class))

# print the confusion matrix for Train
print(metrics.confusion_matrix(y_train, y_pred_train))
print('---------------------')
print(metrics.classification_report(y_train, y_pred_train))

# print the confusion matrix for Test
print(metrics.confusion_matrix(y_test, y_pred_class))
print('------------------------')
print(metrics.classification_report(y_test, y_pred_class))


0.927533912208395
0.8822776199431795
[[18745  3900]
 [ 1533 50795]]
---------------------
              precision    recall  f1-score   support

           0       0.92      0.83      0.87     22645
           1       0.93      0.97      0.95     52328

    accuracy                           0.93     74973
   macro avg       0.93      0.90      0.91     74973
weighted avg       0.93      0.93      0.93     74973

[[ 5684  1933]
 [ 1009 16365]]
------------------------
              precision    recall  f1-score   support

           0       0.85      0.75      0.79      7617
           1       0.89      0.94      0.92     17374

    accuracy                           0.88     24991
   macro avg       0.87      0.84      0.86     24991
weighted avg       0.88      0.88      0.88     24991



In [ ]:
print("XGB Classifier Accuracy:",metrics.accuracy_score(y_test, y_pred_class))

XGB Classifier Accuracy: 0.8822776199431795


**SUPPORT VECTOR MACHINES**

In [ ]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

# instantiate the vectorizer
vect = TfidfVectorizer(max_features=10000)

X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

X_train_dtm

<74973x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 4819607 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.svm import LinearSVC

svm_model = LinearSVC()

# train the model using X_train_dtm
svm_model.fit(X_train_dtm, y_train)

# make class predictions for X_test_dtm
y_pred_class = svm_model.predict(X_test_dtm)

y_pred_train = svm_model.predict(X_train_dtm)

# calculate accuracy of class predictions
from sklearn import metrics
print(metrics.accuracy_score(y_train, y_pred_train))
print(metrics.accuracy_score(y_test, y_pred_class))

# print the confusion matrix for Train
print(metrics.confusion_matrix(y_train, y_pred_train))
print('---------------------')
print(metrics.classification_report(y_train, y_pred_train))

# print the confusion matrix for Test
print(metrics.confusion_matrix(y_test, y_pred_class))
print('------------------------')
print(metrics.classification_report(y_test, y_pred_class))


0.9347098288717272
0.8967228202152775
[[19604  3041]
 [ 1854 50474]]
---------------------
              precision    recall  f1-score   support

           0       0.91      0.87      0.89     22645
           1       0.94      0.96      0.95     52328

    accuracy                           0.93     74973
   macro avg       0.93      0.92      0.92     74973
weighted avg       0.93      0.93      0.93     74973

[[ 6137  1480]
 [ 1101 16273]]
------------------------
              precision    recall  f1-score   support

           0       0.85      0.81      0.83      7617
           1       0.92      0.94      0.93     17374

    accuracy                           0.90     24991
   macro avg       0.88      0.87      0.88     24991
weighted avg       0.90      0.90      0.90     24991



In [ ]:
print("Linear SVM Accuracy:",metrics.accuracy_score(y_test, y_pred_class))

Linear SVM Accuracy: 0.8967228202152775
